# Audio Classification - Audio MNIST dataset

In [1]:
import shutil
from pathlib import Path
import pickle
from collections import Counter, defaultdict

from tqdm import tqdm
import wisardpkg as wp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav

from free_spoken_digit_dataset.utils.fsdd import FSDD
from free_spoken_digit_dataset.utils.spectogramer import dir_to_spectrogram


def load_audio_files(audio_dir):
    sample_rate_list = list()
    samples_list = list()
    names = list()
    files = [x for x in audio_dir.iterdir() if x.is_file()]
    for file in files:
        sample_rate, samples = wav.read(file)
        sample_rate_list.append(sample_rate)
        samples_list.append(samples)
        names.append(file.name)
        
    return sample_rate_list, samples_list, names


def pad_samples(samples, max_n_samples=None):
    if not max_n_samples:
        max_n_samples = np.max([len(samples_i) for samples_i in samples])
    
    padded_samples = [np.append(samples_i, [0]*(max_n_samples - len(samples_i))) 
                      if len(samples_i) < max_n_samples else samples_i[:max_n_samples]  
                      for samples_i in samples]
    return padded_samples


def plot_amplitude(samples, sample_rate, names, i):
    if isinstance(sample_rate, list):
        sample_rate_i = sample_rate[i]
    else:
        sample_rate_i = sample_rate
    
    secs = np.arange(0, len(samples[i])) / sample_rate_i
    
    fig, ax = plt.subplots(figsize=(10,5))
    _ = ax.plot(secs, samples[i])
    _ = ax.set_title(f'Sample {i} - {names[i]}')
    _ = ax.set_xlabel('Seconds')
    _ = ax.set_ylabel('Amplitude')
    plt.show()

## Loading and cleaning data

In [2]:
audio_dir = Path('free_spoken_digit_dataset/recordings/')
sample_rates, samples, names = load_audio_files(audio_dir)

Let's create the labels from the name of the recordings' files.

In [3]:
y = [int(n[:1]) for n in names]
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Now we need to separate the dataset in train and test set.

In [20]:
def split_train_and_test(samples, sample_rates, labels, file_names):
    X_train = list()
    train_sample_rates = list()
    train_file_names = list()
    y_train = list()
    X_test = list()
    test_sample_rates = list()
    test_file_names = list()
    y_test = list()
    
    for i, file_name in enumerate(file_names):
        rec_number = int(file_name.split('_')[-1].split('.')[0])
        if rec_number in [0, 1, 2, 3, 4]:
            X_test.append(samples[i])
            y_test.append(labels[i])
            test_sample_rates.append(sample_rates[i])
            test_file_names.append(file_name)
        else:
            X_train.append(samples[i])
            y_train.append(labels[i])
            train_sample_rates.append(sample_rates[i])
            train_file_names.append(file_name)
    
    return X_train, y_train, X_test, y_test, train_sample_rates, test_sample_rates, train_file_names, test_file_names

X_train, y_train, X_test, y_test,train_sample_rates, test_sample_rates,\
train_file_names, test_file_names = split_train_and_test(samples, sample_rates, y, names)

In [5]:
np.array(X_test).shape

<ipython-input-5-e5f7ff96b72b>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(X_test).shape


(300,)

## Feature Extraction and hyperparameter optimization

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from typing import List, Union, Optional
from time import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.utils.validation import check_is_fitted
from python_speech_features import mfcc
from tqdm import tqdm

np.seterr(divide='ignore')

class DatabaseZScorer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self._mean = None
        self._std = None
    
    def fit(self, x, y=None):
        x = np.concatenate(x, axis=0)
        self._mean = np.mean(x, axis=0)
        self._std = np.std(x, axis=0)
        return self
        
    def transform(self, x):
        x_transformed = list()
        for x_i in x:
            x_i = (x_i - self._mean) / self._std
            x_transformed.append(x_i)
        return x_transformed

    
def extract_features(audio_signals, sample_rates, n_filters_mfcc, window_size, 
                     window_step, append_sum=True, verbose=False):
    features = list()

    if verbose:
        iterator = tqdm(zip(audio_signals, sample_rates), desc='Extracting Features')
    else:
        iterator = zip(audio_signals, sample_rates)
    
    for audio_signal, sample_rate in iterator:
        mfccs = mfcc(audio_signal, samplerate=sample_rate, nfft=1024,
                     winlen=window_size, winstep=window_step, 
                     numcep=n_filters_mfcc, nfilt=n_filters_mfcc, 
                     winfunc=np.hamming)

        if append_sum:
            mfccs_processed = np.cumsum(mfccs, axis=0)
        else:
            mfccs_processed = mfccs

        features.append(mfccs_processed)
        
    return features


def tanh(x):
    return [np.tanh(sample) for sample in x]


def cross_validation_wisard(audio_recordings, y, sample_rates, file_names, n_splits=5, addressSize=[50], 
                            bleachingActivated=[True], n_filters_mfcc=[20], 
                            window_size=[0.05], window_step=[0.05], append_sum=[True], 
                            n_kernels=[10]):
    grid_search_results = []
    best_params = None
    best_accuracy = -1
    
    if not isinstance(y, np.ndarray):
        y = np.array(y)
    
    param_grid_dict = dict()
    param_grid_dict['addressSize'] = addressSize
    param_grid_dict['bleachingActivated'] = bleachingActivated
    param_grid_dict['n_filters_mfcc'] = n_filters_mfcc
    param_grid_dict['window_size'] = window_size
    param_grid_dict['window_step'] = window_step
    param_grid_dict['append_sum'] = append_sum
    param_grid_dict['n_kernels'] = n_kernels
    
    stratification_groups = [name.rsplit('_', 1)[0] for name in file_names] # by label and speaker
    skf = StratifiedKFold(n_splits=n_splits)
    
    param_grid = ParameterGrid(param_grid_dict)
    
    param_grid_combinations = len(list(param_grid))
    n_combs = n_splits * param_grid_combinations
    progress = tqdm(total=n_combs, position=0, leave=True)
    
    for params in param_grid:
        print(params)
        preprocessing_time = []
        training_time = []
        training_prediction_time = []
        validation_prediction_time = []
        training_accuracy = []
        validation_accuracy = []
        for train_index, val_index in skf.split(np.zeros(len(audio_recordings)), stratification_groups):
            X_train = [audio_recordings[i] for i in train_index]
            y_train = y[train_index]
            y_train_str = [str(l) for l in y_train]
            
            X_val = [audio_recordings[i] for i in val_index]
            y_val = y[val_index]
            y_val_str = [str(l) for l in y_val]
            
            # Preprocessing
            start = time()
            
            # 1) Extract MFCCs from audio recordings and use Appendsum (or not)
            X_train = extract_features(X_train, sample_rates, params['n_filters_mfcc'], params['window_size'], 
                                       params['window_step'], params['append_sum'], verbose=False)
            X_val = extract_features(X_val, sample_rates, params['n_filters_mfcc'], params['window_size'], 
                                     params['window_step'], params['append_sum'], verbose=False)
            
            # 2) Database Z-score
            z_scorer = DatabaseZScorer()
            X_train = z_scorer.fit_transform(X_train)
            X_val = z_scorer.transform(X_val)
            
            # 3) Hyperbolic tangent (scaling features to (-1, +1) interval
            X_train = tanh(X_train)
            X_val = tanh(X_val)
            
            # 4) Applying kernel canvas to each recordings' feature set
            dimension = X_train[0].shape[1]
            kernel_canvas = wp.KernelCanvas(dimension, params['n_kernels'], bitsBykernel = 16, 
                                            activationDegree = 0.075)
            
            X_train = [kernel_canvas.transform(rec_features) for rec_features in X_train]
            X_val = [kernel_canvas.transform(rec_features) for rec_features in X_val]
            
            end = time()
            preprocessing_time.append(end-start)
            
            # training
            model = wp.Wisard(params['addressSize'], bleachingActivated=params['bleachingActivated'],
                              ignoreZero=True)
            
            start = time()
            model.train(X_train, y_train_str)
            end = time()
            training_time.append(end-start)
            
            start = time()
            pred_train = model.classify(X_train)
            end = time()
            training_prediction_time.append(end-start)
            
            start = time()
            pred_val = model.classify(X_val)
            end = time()
            validation_prediction_time.append(end-start)
            
            training_accuracy.append(accuracy_score(y_train_str, pred_train))
            
            validation_accuracy.append(accuracy_score(y_val_str, pred_val))
            
            progress.update()
        
        cv_result = {'preprocessing_time_mean': np.mean(preprocessing_time),
                     'preprocessing_time_std': np.std(preprocessing_time),
                     'training_time_mean': np.mean(training_time),
                     'training_time_std': np.std(training_time),
                     'training_prediction_time_mean': np.mean(training_prediction_time),
                     'training_prediction_time_std': np.std(training_prediction_time),
                     'validation_prediction_time_mean': np.mean(validation_prediction_time),
                     'validation_prediction_time_std': np.std(validation_prediction_time),
                     'training_accuracy_mean': np.mean(training_accuracy),
                     'training_accuracy_std': np.std(training_accuracy),
                     'validation_accuracy_mean': np.mean(validation_accuracy),
                     'validation_accuracy_std': np.std(validation_accuracy)}
        
        cv_result = {**params, **cv_result}
        
        grid_search_results.append(cv_result)
        
        if (cv_result['validation_accuracy_mean'] > best_accuracy):
            best_accuracy = cv_result['validation_accuracy_mean']
            best_params = params
        
        print(f"Train acc. mean: {cv_result['training_accuracy_mean']:.4f} | Validation acc. mean: {cv_result['validation_accuracy_mean']:.4f}\n")
    
    grid_search_results = pd.DataFrame(grid_search_results)
    grid_search_results['rank'] = grid_search_results['validation_accuracy_mean'].rank(ascending=False)
    
    return grid_search_results, best_params

In [35]:
ws_gs_results, ws_best_params = cross_validation_wisard(X_train, y_train, sample_rates, train_file_names, n_splits=5, 
                                                        addressSize=[5, 10, 15, 20], bleachingActivated=[True], 
                                                        n_filters_mfcc=[10, 12, 14, 16], window_size=[0.1], window_step=[0.02], 
                                                        append_sum=[True], n_kernels=[2**10, 2**11, 2**12, 2**13])

  0%|          | 0/320 [00:00<?, ?it/s]

{'addressSize': 5, 'append_sum': True, 'bleachingActivated': True, 'n_filters_mfcc': 10, 'n_kernels': 1024, 'window_size': 0.1, 'window_step': 0.02}


  2%|▏         | 5/320 [01:25<1:29:04, 16.97s/it]

Train acc. mean: 0.9262 | Validation acc. mean: 0.7693

{'addressSize': 5, 'append_sum': True, 'bleachingActivated': True, 'n_filters_mfcc': 10, 'n_kernels': 2048, 'window_size': 0.1, 'window_step': 0.02}


  3%|▎         | 10/320 [03:31<2:01:52, 23.59s/it]

...

Train acc. mean: 1.0000 | Validation acc. mean: 0.8515

{'addressSize': 20, 'append_sum': True, 'bleachingActivated': True, 'n_filters_mfcc': 16, 'n_kernels': 4096, 'window_size': 0.1, 'window_step': 0.02}


 98%|█████████▊| 315/320 [3:31:57<03:10, 38.05s/it]

Train acc. mean: 1.0000 | Validation acc. mean: 0.8774

{'addressSize': 20, 'append_sum': True, 'bleachingActivated': True, 'n_filters_mfcc': 16, 'n_kernels': 8192, 'window_size': 0.1, 'window_step': 0.02}


100%|██████████| 320/320 [3:38:35<00:00, 72.91s/it]

Train acc. mean: 1.0000 | Validation acc. mean: 0.8907



In [40]:
ws_gs_results = ws_gs_results.sort_values('rank').reset_index(drop=True)

with open('results/wisard2_gs_results.pickle', 'wb') as file:
    pickle.dump(ws_gs_results, file)

ws_gs_results

,addressSize,append_sum,bleachingActivated,n_filters_mfcc,n_kernels,window_size,window_step,preprocessing_time_mean,preprocessing_time_std,training_time_mean,training_time_std,training_prediction_time_mean,training_prediction_time_std,validation_prediction_time_mean,validation_prediction_time_std,training_accuracy_mean,training_accuracy_std,validation_accuracy_mean,validation_accuracy_std,rank
0,10,True,True,16,8192,0.1,0.02,68.779536,0.461718,1.947594,0.046322,7.282843,0.064355,1.763058,0.011880,1.000000,0.000000,0.914074,0.025272,1.0
1,10,True,True,14,8192,0.1,0.02,68.809825,2.102739,2.021882,0.132910,7.123245,0.131645,1.704458,0.026431,1.000000,0.000000,0.912963,0.009443,2.0
2,10,True,True,16,2048,0.1,0.02,21.257074,0.148606,0.390748,0.010244,1.612594,0.060409,0.457569,0.041106,1.000000,0.000000,0.912222,0.022999,3.0
3,10,True,True,12,8192,0.1,0.02,67.674637,1.687225,2.156996,0.451407,7.586651,1.009147,1.688890,0.027610,1.000000,0.000000,0.911481,0.009544,4.0
4,15,True,True,14,8192,0.1,0.02,69.580245,3.124243,1.803700,0.085118,6.189355,0.109496,1.501977,0.040649,1.000000,0.000000,0.911481,0.016580,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,5,True,True,16,2048,0.1,0.02,21.195311,0.260166,0.380976,0.035190,2.672261,0.162582,0.684966,0.069627,0.965556,0.003135,0.786296,0.024612,60.0
60,5,True,True,12,1024,0.1,0.02,20.212808,2.871695,0.213629,0.031466,1.580484,0.401744,0.400684,0.126144,0.944444,0.005834,0.779259,0.018310,61.0
61,5,True,True,10,1024,0.1,0.02,15.484381,1.137779,0.166962,0.023644,1.181425,0.172768,0.308179,0.040861,0.926204,0.007463,0.769259,0.023733,62.0
62,5,True,True,14,1024,0.1,0.02,14.111317,0.126468,0.158926,0.001679,1.118401,0.016352,0.290834,0.010492,0.935185,0.009676,0.768519,0.015135,63.0


In [41]:
with open('results/wisard2_gs_best_params.pickle', 'wb') as file:
    pickle.dump(ws_best_params, file)
    
ws_best_params

{'addressSize': 10,
 'append_sum': True,
 'bleachingActivated': True,
 'n_filters_mfcc': 16,
 'n_kernels': 8192,
 'window_size': 0.1,
 'window_step': 0.02}

In [38]:
with open('results/wisard2_gs_best_params.pickle', 'rb') as file:
    ws_best_params = pickle.load(file)

In [39]:
# Preprocessing
y_train_str = [str(l) for l in y_train]
y_test_str = [str(l) for l in y_test]

# 1) Extract MFCCs from audio recordings and use Appendsum (or not)
X_train_preprocessed = extract_features(X_train, sample_rates, ws_best_params['n_filters_mfcc'], ws_best_params['window_size'], 
                                        ws_best_params['window_step'], ws_best_params['append_sum'], verbose=False)
X_test_preprocessed = extract_features(X_test, sample_rates, ws_best_params['n_filters_mfcc'], ws_best_params['window_size'], 
                                       ws_best_params['window_step'], ws_best_params['append_sum'], verbose=False)

# 2) Database Z-score
z_scorer = DatabaseZScorer()
X_train_preprocessed = z_scorer.fit_transform(X_train_preprocessed)
X_test_preprocessed = z_scorer.transform(X_test_preprocessed)

# 3) Hyperbolic tangent (scaling features to (-1, +1) interval
X_train_preprocessed = tanh(X_train_preprocessed)
X_test_preprocessed = tanh(X_test_preprocessed)

# 4) Applying kernel canvas to each recordings' feature set
dimension = X_train_preprocessed[0].shape[1]
kernel_canvas = wp.KernelCanvas(dimension, ws_best_params['n_kernels'], 
                                bitsBykernel = 16, activationDegree = 0.075)

X_train_preprocessed = [kernel_canvas.transform(rec_features) for rec_features in X_train_preprocessed]
X_test_preprocessed = [kernel_canvas.transform(rec_features) for rec_features in X_test_preprocessed]

# training
model = wp.Wisard(ws_best_params['addressSize'], 
                  bleachingActivated=ws_best_params['bleachingActivated'],
                  ignoreZero=True)

model.train(X_train_preprocessed, y_train_str)

pred_train = model.classify(X_train_preprocessed)
pred_test = model.classify(X_test_preprocessed)

training_accuracy = accuracy_score(y_train_str, pred_train)
test_accuracy = accuracy_score(y_test_str, pred_test)

print(f'Training Accuracy: {training_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Training Accuracy: 1.0000
Test Accuracy: 0.9467


## Results visualization

In [ ]:
with open('results/wisard2_gs_results.pickle', 'rb') as file:
    ws_gs_results = pickle.load(file)

ws_gs_results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(5,8))
ax = sns.boxplot(x="n_kernels", y="validation_accuracy_mean", data=ws_gs_results, ax=ax)
ax.set_ylabel('Validation Accuracy')
ax.set_title('5-folds CV results')